## Flask

- Super useful framework
- With a few lines of code, we easily create an endpoint
- Documentation here: http://flask.pocoo.org/docs/0.10/

In [ ]:
# pip install Flask (your machines should already have it installed)

from flask import Flask # here we import the Flask class
app = Flask(__name__) # we create an instance of the Flask class -> '__name__'

@app.route("/") # we use whats called a 'decorator' to tell Flask what URL should trigger our function
def hello():
    return "Hello World!" 


# only runs if the script is executed directly from the Python interpreter (and not used as an imported module)
if __name__ == "__main__":  

    app.debug = True # we want to get error messages
    app.run(host='0.0.0.0', port=8886) # make your endpoint publicly available 

- we move this code to its own file
- then we run it on our remote machine (execute it directly from the Python interpreter)
- remember to open up the port 
- and now we can see its results when we navigate to:
    - http://[YOUR_PUBLIC_DNS]:8886/

- HELLO WORLD!

In [ ]:
# let's add a function -> given a tag, return the most recent instagram image for the tag

@app.route('/igtag/<tag>')
def ig_recent_tag(tag):
    
    # here we write our function that calls instagram and grabs the latest IG media per tag
    
    
    
    
